# Battle of the Neighborhood

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this project we will find the best neighborhood to open a theater in Chicago.

Since there are theaters throughout the city, the goal is to find the neighboor with a high popluation, low amount of theaters, and use clustering to find Neighborhoods that are similar to the neighborhoods that already have movie theaters.

A list of top neighboors will be provided. Allowing them to see pros and cons from each neighborhood to decide where to open a new theater.

## Data <a name="data"></a>

Looking at the problem the key data need:


* The popluation of Neighborhoods
* The amount of theaters in the Neighborhoods
* The amount of attracts in the Neighborhoods
* The Area of the Neighborhood
* The Popluation Density of a Neighborhood


### Grabbing the Popluation and Area per Neighborhood

This is the libraries used for the project.

In [93]:
import numpy as np
import pandas as pd

import json
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors

from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
import folium

This Section is web scraping the neighborhoods, Popluation, Area, and Popluation Density.

In [101]:
url = "https://en.wikipedia.org/wiki/Community_areas_in_Chicago"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, "html5lib")

The newest population by neighborhood that was found was 2017. This data will be used for the population and population density.

In [102]:
Neighborhood_data = pd.DataFrame(columns=['Number', 'Name', '2017 Population', 'Area (Sq mi.)', 'Area (km^2)', '2017 Population Density (Sq mi.)', '2017 Population Density (km^2)'])

tables = soup.find("table")
for rows in tables.findAll("tr"):
  col = rows
  col = col.text.split("\n\n")
  num = col[0].replace('\n', '')
  name = col[1]
  pop = col[2]
  a1 = col[3]
  a2 = col[4]
  pd1 = col[5]
  pd2 = col[6].replace('\n', '')
  
  Neighborhood_data = Neighborhood_data.append({"Number":num, "Name":name, "2017 Population":pop, "Area (Sq mi.)":a1, "Area (km^2)":a2, "2017 Population Density (Sq mi.)":pd1, "2017 Population Density (km^2)":pd2}, ignore_index=True)


Cleaning up the DataFrame up.

In [103]:
Neighborhood_data = Neighborhood_data[1:-1]
Neighborhood_data

,Number,Name,2017 Population,Area (Sq mi.),Area (km^2),2017 Population Density (Sq mi.),2017 Population Density (km^2)
1,01,Rogers Park,"55,062",1.84,4.77,"29,925.00","11,554.11"
2,02,West Ridge,"76,215",3.53,9.14,"21,590.65","8,336.20"
3,03,Uptown,"57,973",2.32,6.01,"24,988.36","9,648.06"
4,04,Lincoln Square,"41,715",2.56,6.63,"16,294.92","6,291.50"
5,05,North Center,"35,789",2.05,5.31,"17,458.05","6,740.59"
...,...,...,...,...,...,...,...
73,73,Washington Heights,"27,453",2.86,7.41,"9,598.95","3,706.18"
74,74,Mount Greenwood,"19,277",2.71,7.02,"7,113.28","2,746.45"
75,75,Morgan Park,"22,394",3.30,8.55,"6,786.06","2,620.11"
76,76,O'Hare,"12,377",13.34,34.55,927.81,358.23


### Latitudes and Longitudes

This section is getting the latitudes and longitudes for each Neighborhood.

Using a excel has the zip codes latitude and longitude and this section puts it in a DataFrame.

In [157]:
lat_lng_data = pd.read_excel("us-zip-code-latitude-and-longitude.xlsx")
lat_lng_data = lat_lng_data.drop(labels=['City','State', 'Timezone', 'Daylight savings time flag', 'geopoint'], axis=1)
lat_lng_data

,Zip,Latitude,Longitude
0,60415,41.702482,-87.77869
1,60668,41.811929,-87.68732
2,60646,41.995331,-87.76010
3,60654,41.888627,-87.63538
4,60647,41.921126,-87.70085
...,...,...,...
87,60411,41.511307,-87.61010
88,60678,41.811929,-87.68732
89,60671,41.811929,-87.68732
90,60681,41.811929,-87.68732


This web scraps the website to get the zip code for each neighborhood.

In [98]:
url ="https://www.chicagotribune.com/chi-community-areas-htmlstory.html"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, "html5lib")
tables = soup.find_all("table")

zip_code = pd.DataFrame(columns=['Zip', 'Name'])
i = 0
for table in tables:
  if i == 3:
    for rows in table.findAll("tr"):
      col = rows.find_all("td")
      zipc = col[0].text
      location = []
      location = col[1].text.split(", ")
      if len(location) == 1:
        zip_code = zip_code.append({"Zip":int(zipc), "Name":str(location[0])}, ignore_index=True)

      if len(location) > 1:
        for i in range(len(location)):
          zip_code = zip_code.append({"Zip":int(zipc), "Name":str(location[i])}, ignore_index=True)

  i +=1 

zip_code.head()

,Zip,Name
0,60601,Loop
1,60602,Loop
2,60603,Loop
3,60604,Loop
4,60605,Loop


This section merge the two DataFrames on the zip code column. It then drops the zip code column. Instead of having mutiple latitudes and longitudes the DataFrame is groupby and average to get one latitude and longitude for each neighborhood.

In [204]:
zip_code_neigh = pd.merge(zip_code, lat_lng_data, on="Zip")
zip_code_neigh = zip_code_neigh.drop(['Zip'], axis=1)
n_lat_lng = zip_code_neigh.groupby("Name").mean().reset_index()
n_lat_lng = n_lat_lng.sort_values(by=['Name'], ascending=True)
n_lat_lng.head()

,Name,Latitude,Longitude
0,Albany Park,41.971329,-87.730625
1,Archer Heights,41.809299,-87.710500
2,Armour Square,41.830842,-87.642040
3,Ashburn,41.744600,-87.711880
4,Auburn Gresham,41.740483,-87.652820


This is cleaning up the Neighborhood data and getting a new DataFrame that will be used to merge the Chicago neighborhoods and their latitudes and longitudes.

In [288]:
Neighborhood_data['Name'] = Neighborhood_data['Name'].str.replace('(The)','')
Neighborhood_data['Name'] = Neighborhood_data['Name'].str.replace("11",'')
Neighborhood_data['Name'] = Neighborhood_data['Name'].str.strip("[]()")  	
Neighborhood_data['Name'] = Neighborhood_data['Name'].str.replace("Beverly Hills",'Beverly')
Neighborhood_data['Upper'] = Neighborhood_data['Name'].str.upper()
Neighborhood_data['Upper'] = Neighborhood_data['Upper'].str.replace(' ','')
Neighborhood_data['Upper'] = Neighborhood_data['Upper'].str.replace('BEVERLYHILLS','BEVERLY')
merge1 = Neighborhood_data.drop(['2017 Population', 'Area (Sq mi.)', 'Area (km^2)', '2017 Population Density (Sq mi.)', '2017 Population Density (km^2)'], axis=1)
merge1

,Number,Name,Upper
1,01,Rogers Park,ROGERSPARK
2,02,West Ridge,WESTRIDGE
3,03,Uptown,UPTOWN
4,04,Lincoln Square,LINCOLNSQUARE
5,05,North Center,NORTHCENTER
...,...,...,...
73,73,Washington Heights,WASHINGTONHEIGHTS
74,74,Mount Greenwood,MOUNTGREENWOOD
75,75,Morgan Park,MORGANPARK
76,76,O'Hare,O'HARE


This section is cleaning up the latitude and longitude DataFrame and creating another DataFrame for merging Neighborhood and latitude and longitude. The Riverdale information is added, because it was missing.

In [205]:
n_lat_lng['Upper'] = n_lat_lng['Name'].str.upper()
n_lat_lng['Upper'] = n_lat_lng['Upper'].str.replace(' ','')
n_lat_lng['Upper'] = n_lat_lng['Upper'].str.replace('(PILSEN)','').str.strip("[]()")
merge2 = n_lat_lng.drop('Name', axis=1)
merge2 = merge2.append({'Latitude':41.66, 'Longitude':-87.61, 'Upper':'RIVERDALE'}, ignore_index=True)
merge2.sort_values(by='Upper').head()

,Latitude,Longitude,Upper
0,41.971329,-87.730625,ALBANYPARK
1,41.809299,-87.710500,ARCHERHEIGHTS
2,41.830842,-87.642040,ARMOURSQUARE
3,41.744600,-87.711880,ASHBURN
4,41.740483,-87.652820,AUBURNGRESHAM


Finally merging the two and dropping entries in the DataFrame that don't have neighborhoods. Now we have data to use for FourSquare.

In [207]:
result = pd.merge(merge1, merge2, how="left", on=["Upper"]).dropna()
result = result.drop(['Name','Upper'],axis=1)
num_name = Neighborhood_data[['Number', 'Name']]
result = result.groupby('Number').mean()
Neigh_lat_lng = pd.merge(num_name, result, how="left", on=["Number"])
Neigh_lat_lng = Neigh_lat_lng.drop('Number',axis=1)
Neigh_lat_lng

,Name,Latitude,Longitude
0,Rogers Park,42.009731,-87.669380
1,West Ridge,42.000169,-87.700060
2,Uptown,41.963218,-87.664700
3,Lincoln Square,41.972397,-87.684530
4,North Center,41.945150,-87.671185
...,...,...,...
72,Washington Heights,41.710783,-87.645100
73,Mount Greenwood,41.696283,-87.699120
74,Morgan Park,41.696358,-87.679525
75,O'Hare,41.975080,-87.816300


### Foursquare


##### Getting data from FourSquare

In [210]:
CLIENT_ID = '' 
CLIENT_SECRET = ''
VERSION = '20180605' # Foursquare API version
LIMIT = 100 
radius = 500

A function the will get the information from FourSquare and store it into a DataFrame.

In [216]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [217]:
Venue_data = getNearbyVenues(names=Neigh_lat_lng['Name'], latitudes=Neigh_lat_lng['Latitude'], longitudes=Neigh_lat_lng['Longitude'])
Venue_data

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rogers Park,42.009731,-87.66938,Morse Fresh Market,42.008087,-87.667041,Grocery Store
1,Rogers Park,42.009731,-87.66938,The Common Cup,42.007797,-87.667901,Coffee Shop
2,Rogers Park,42.009731,-87.66938,Smack Dab,42.009291,-87.666201,Bakery
3,Rogers Park,42.009731,-87.66938,Glenwood Sunday Market,42.008525,-87.666251,Farmers Market
4,Rogers Park,42.009731,-87.66938,Rogers Park Social,42.007360,-87.666265,Bar
...,...,...,...,...,...,...,...
1307,Edgewater,41.981906,-87.66660,Orange Shoe Personal Fitness,41.983753,-87.668974,Gym / Fitness Center
1308,Edgewater,41.981906,-87.66660,Found Vintage Living,41.983545,-87.669328,Antique Shop
1309,Edgewater,41.981906,-87.66660,Oda Mediterranean Cuisine,41.984993,-87.668915,Restaurant
1310,Edgewater,41.981906,-87.66660,CTA Bus Stop 17216,41.985350,-87.669300,Bus Station


#### Cleaning up the data

In [221]:
Venue_data.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Loop,66,66,66,66,66,66
Albany Park,24,24,24,24,24,24
Archer Heights,14,14,14,14,14,14
Armour Square,20,20,20,20,20,20
Ashburn,4,4,4,4,4,4
...,...,...,...,...,...,...
West Lawn,8,8,8,8,8,8
West Pullman,11,11,11,11,11,11
West Ridge,21,21,21,21,21,21


This is creating a onehot encoder to be used for clustering.

In [227]:
Neighborhood_onehot = pd.get_dummies(Venue_data[['Venue Category']], prefix="", prefix_sep="")
Neighborhood_onehot['Neighborhood'] = Venue_data['Neighborhood']
fixed_columns = [Neighborhood_onehot.columns[-1]] + list(Neighborhood_onehot.columns[:-1])
Neighborhood_onehot = Neighborhood_onehot[fixed_columns]
pd.set_option('display.max_columns', None) 
Neighborhood_onehot.head()

,Yoga Studio,ATM,Accessories Store,Adult Boutique,Airport,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bed & Breakfast,Beer Bar,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Boutique,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Café,Camera Store,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Quad,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Public Art,Record Shop,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Snack Place,Soccer Field,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Storage Facility,Student Center,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Toy / Game Store,Trail,Train Station,Transportation Service,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rogers Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

This is grouping and getting all the data with the neighborhoods with movie theaters. Theaters are added because there isn't away to break the difference between movie theaters and play theaters.

In [265]:
Neigh_indiemovie_count = Neighborhood_onehot[['Neighborhood', 'Indie Movie Theater']]
Neigh_indiemovie_count = Neigh_indiemovie_count.loc[(Neigh_indiemovie_count['Indie Movie Theater']!=0)]
Neigh_movie_count = Neighborhood_onehot[['Neighborhood', 'Movie Theater']]
Neigh_movie_count = Neigh_movie_count.loc[(Neigh_movie_count['Movie Theater']!=0)]
Neigh_theater_count = Neighborhood_onehot[['Neighborhood', 'Theater']]
Neigh_theater_count = Neigh_theater_count.loc[(Neigh_theater_count['Theater']!=0)]
Neigh_withtheater = Neigh_indiemovie_count.append(Neigh_movie_count, sort=False)
Neigh_withtheater = Neigh_withtheater.append(Neigh_theater_count, sort=False)
Neigh_withtheater = Neigh_withtheater.groupby('Neighborhood').count()
Neigh_withtheater

,Indie Movie Theater,Movie Theater,Theater
Neighborhood,,,
Loop,0,0,2
Chatham,0,1,0
Edgewater,0,0,1
Hyde Park,0,0,1
Lake View,1,0,1
Near West Side,0,0,1
North Center,0,0,1
Rogers Park,0,0,2
South Lawndale,0,0,1


This is will be used to Cluster the Neighborhoods. Takeing out the movie theater to make sure they are cluster by it. The popluation will be added to have another dimension to cluster the neighborhoods.

In [287]:
Neigh_cluster = Neighborhood_onehot.drop(['Indie Movie Theater', 'Movie Theater', 'Theater'], axis =1)
Neigh_group = Neigh_cluster.groupby('Neighborhood').mean().reset_index()
N_pop = Neighborhood_data[['Name', '2017 Population']]
N_pop = N_pop.rename(columns={"Name": "Neighborhood", "2017 Population":'Pop.'})
Neigh_group_f = pd.merge(Neigh_group, N_pop, how="left", on=["Neighborhood"]).dropna()
Neigh_group_f

,Neighborhood,Yoga Studio,ATM,Accessories Store,Adult Boutique,Airport,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bed & Breakfast,Beer Bar,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Boutique,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,...,Seafood Restaurant,Shipping Store,Shoe Store,Snack Place,Soccer Field,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Storage Facility,Student Center,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Thrift / Vintage Store,Tour Provider,Toy / Game Store,Trail,Train Station,Transportation Service,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Pop.
0,Loop,0.0,0.0,0.0,0.0,0.0,0.0,0.030303,0.0,0.0,0.000000,0.0,0.015152,0.015152,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.030303,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015152,0.0,0.000000,0.000000,0.0,0.0,0.015152,0.015152,...,0.000000,0.0,0.015152,0.030303,0.0,0.0,0.0,0.000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.015152,0.0,0.015152,0.0,0.000000,0.0,0.0,0.0,0.0,0.015152,0.000000,0.000000,0.030303,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,"35,880"
1,Albany Park,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.041667,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.041667,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.0,0.041667,0.0,0.0,0.000000,0.0,0.000000,0.0,0.041667,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,"51,992"
2,Archer Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.071429,0.000000,0.000000,0.071429,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.285714,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000,0.000000,0.00,0.00,0.0,0.000000,0.071429,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.071429,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,"13,142"
3,Armour Square,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.050000,0.050000,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.050,0.000000,0.05,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.0,"13,455"
4,Ashburn,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000,0.000000,0.00,0.25,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.250000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,"43,792"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,West Lawn,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.250000,0.000000,0.000000,0.0,0.0

## Methodology <a name="methodology"></a>

## Analysis <a name="analysis"></a>

## Results and Discussion <a name="results"></a>

## Conclusion <a name="conclusion"></a>